### Notebook name: ImageShareabilityClassifiers.ipynb
#### Author: Sreejith Menon (smenon8@uic.edu)

### General Description:
Multiple features are extracted per image.    
The features are majorly classified as:
* Bilogical features like age, species, sex
* Ecological features like yaw, view_point
* Image EXIF/Quality data: unixtime, latitude, longitude, quality
* Tags generated by Microsoft Image tagging API
* Image Contributor - Sparse attribute
* Individual animals (NID)

Based on these features mutliple classification algorithms are implemented and the metrics are evaluated. The aim of the classification algorithms is to predict given features, will a certain image be shared/not shared on a social media platform.    
The ClassifierHelperAPI has *off-the-shelf* implementations from `sk-learn` library and uses a Classifier Object to store the metrics of each classifier.    
The performance metrics evaluated are:
* Accuracy - Number of correct predictions in the test data
* Precision 
* Recall
* F1 score
* Absolute Error
* AUC
* Squared Error - Not displayed currently
* Zero One Hinge Loss - Not displayed currently

In [255]:
import ClassiferHelperAPI as CH
import importlib
import numpy as np
import pandas as pd
importlib.reload(CH)
from ast import literal_eval
import plotly.plotly as py
import htmltag as HT
import cufflinks as cf # this is necessary to link pandas to plotly
cf.go_online()
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import Counter
import csv
import plotly.graph_objs as go

### Building data for the Classifier
* Discretizing non-binary data using the bag-of-words model
* Building and running the classifer for all train-test splits starting from 10% upto 90%
* Computing the performance metrics for each of the classifier.

In [161]:
allAttribs = CH.genAllAttribs("../FinalResults/ImgShrRnkListWithTags.csv","sparse","../data/infoGains.csv")
data= CH.getMasterData("../FinalResults/ImgShrRnkListWithTags.csv")

In [162]:
# Block of code for building and running the classifier
# Will generate custom warnings, setting scores to 0, if there are no valid predictions
methods = ['logistic','svm','dtree','random_forests','ada_boost']
methods = ['ada_boost']

classifiers = []
for method in methods:
    for i in np.arange(0.4,0.5,0.1):
        clfObj = CH.buildBinClassifier(data,allAttribs,1-i,80,method)
        clfObj.runClf()
        classifiers.append(clfObj)

TypeError: ABCMeta object argument after ** must be a mapping, not NoneType

In [ ]:
# Writing all the scores into a pandas data-frame and then into a CSV file
printableClfs = []

for clf in classifiers:
    printableClfs.append(dict(literal_eval(clf.__str__())))
    
df = pd.DataFrame(printableClfs)
df = df[['methodName','splitPercent','accScore','precision','recall','f1Score','auc','sqerr']]
df.columns = ['Classifier','Train-Test Split','Accuracy','Precision','Recall','F1 score','AUC','Squared Error']
# df.to_csv("../ClassifierResults/extrmClfMetrics_abv_mean.csv",index=False)

In [ ]:
# Will take up valuable Plot.ly plots per day. Limited to 50 plots per day.
# changes to file name important
iFrameBlock = []
for i in np.arange(0.4,0.5,0.1):
    df1 = df[(df['Train-Test Split']==1-i)]
    df1.index = df1['Classifier']
    df1 = df1[['Accuracy','Precision','Recall','F1 score','AUC','Squared Error']].transpose()
    df1.iplot(kind='bar',filename=str('Train-Test_Split_Ratio_abv_mean %f' %i),title=str('Train-Test Split Ratio: %f' %i))
    # iFrameBlock.append(fig.embed_code)

# with open("../ClassifierResults/performanceComparisonsparse.html","w") as perf:
#     perf.write(HT.h1("Performance Comparisons of Classifiers with non_sparse Attributes."))
#     for row in iFrameBlock:
#         perf.write(HT.HTML(row))

### Calculating weights of features in the classifiers

In [ ]:
clfWeights = []
for clf in classifiers:
    clfAttribs = list(clf.test_x.columns)
    if clf.methodName == 'logistic':
        clfAttribWgts = list(clf.clfObj.coef_[0])
    elif clf.methodName == 'dtree' or clf.methodName == 'random_forests':
        clfAttribWgts = list(clf.clfObj.feature_importances_)
    else:
        continue
        
        
    attribWgt = {clfAttribs[i] : clfAttribWgts[i] for i in range(len(clfAttribs))}
    attribWgt['Method'] = clf.methodName
    attribWgt['Split_Percent'] = clf.splitPercent
        
    clfWeights.append(attribWgt)

In [ ]:
clfDf = pd.DataFrame(clfWeights)

In [ ]:
indDF = clfDf[(clfDf['Method']=='logistic')]
indDF.index = indDF['Split_Percent']
indDF.drop('Method',1,inplace=True)  
indDF.transpose().to_csv("../ClassifierResults/LogisiticWeights.csv")

indDF = clfDf[(clfDf['Method']=='dtree')]
indDF.index = indDF['Split_Percent']
indDF.drop('Method',1,inplace=True)  
indDF.transpose().to_csv("../ClassifierResults/DecisionTreeWeights.csv")

indDF = clfDf[(clfDf['Method']=='random_forests')]
indDF.index = indDF['Split_Percent']
indDF.drop('Method',1,inplace=True)  
indDF.transpose().to_csv("../ClassifierResults/RandomForestsWeights.csv")

In [ ]:
logisticDf = clfDf[(clfDf['Method']=='logistic')]
del logisticDf['Method']
del logisticDf['Split_Percent']
dtreeDf = clfDf[(clfDf['Method']=='dtree')]
del dtreeDf['Method']
del dtreeDf['Split_Percent']
randomForestDf = clfDf[(clfDf['Method']=='random_forests')]
del randomForestDf['Method']
del randomForestDf['Split_Percent']

In [ ]:
logisticDf = logisticDf.transpose()
logisticDf.reset_index(inplace=True)
logisticDf.columns = ['Feature','10%','20%','30%','40%','50%','60%','70%','80%','90%']
dfs_logistic = []
for i in range(10,100,10):
    prcnt = str(i)+'%'
    logisticDf.sort_values(by=prcnt,inplace=True,ascending=False)
    df = logisticDf[['Feature',prcnt]].head(15)
    df.index = np.arange(1,16,1)
    
    dfs_logistic.append(df)
    
concatdf_logisitc = pd.concat([dfs_logistic[0],dfs_logistic[1],dfs_logistic[2],dfs_logistic[3],dfs_logistic[4],dfs_logistic[5],dfs_logistic[6],dfs_logistic[7],dfs_logistic[8]],axis=1)
concatdf_logisitc.to_csv("../ClassifierResults/Top15_Weights_Logisitic.csv")

In [ ]:
dtreeDf = dtreeDf.transpose()
dtreeDf.reset_index(inplace=True)
dtreeDf.columns = ['Feature','10%','20%','30%','40%','50%','60%','70%','80%','90%']
dfs_tree = []
for i in range(10,100,10):
    prcnt = str(i)+'%'
    dtreeDf.sort_values(by=prcnt,inplace=True,ascending=False)
    df = dtreeDf[['Feature',prcnt]].head(15)
    df.index = np.arange(1,16,1)
    
    dfs_tree.append(df)
    
concatdf_dtree = pd.concat([dfs_tree[0],dfs_tree[1],dfs_tree[2],dfs_tree[3],dfs_tree[4],dfs_tree[5],dfs_tree[6],dfs_tree[7],dfs_tree[8]],axis=1)
concatdf_dtree.to_csv("../ClassifierResults/Top15_Weights_Dtree.csv")

In [ ]:
randomForestDf = randomForestDf.transpose()
randomForestDf.reset_index(inplace=True)
randomForestDf.columns = ['Feature','10%','20%','30%','40%','50%','60%','70%','80%','90%']
dfs_rndf = []
for i in range(10,100,10):
    prcnt = str(i)+'%'
    randomForestDf.sort_values(by=prcnt,inplace=True,ascending=False)
    df = randomForestDf[['Feature',prcnt]].head(15)
    df.index = np.arange(1,16,1)
    
    dfs_rndf.append(df)
    
concatdf_rndf = pd.concat([dfs_rndf[0],dfs_rndf[1],dfs_rndf[2],dfs_rndf[3],dfs_rndf[4],dfs_rndf[5],dfs_rndf[6],dfs_rndf[7],dfs_rndf[8]],axis=1)
concatdf_rndf.to_csv("../ClassifierResults/Top15_Weights_Rndf.csv")

In [ ]:
attribs = [list(dfs_logistic[i]['Feature']) for i in range(0,9)]
attribs = [attrib for listAttrib in attribs for attrib in listAttrib]
pd.DataFrame(Counter(attribs),index=['Frequency']).transpose().sort_values(by=['Frequency'],ascending=False)

In [ ]:
attribs = [list(dfs_tree[i]['Feature']) for i in range(0,9)]
attribs = [attrib for listAttrib in attribs for attrib in listAttrib]
pd.DataFrame(Counter(attribs),index=['Frequency']).transpose().sort_values(by=['Frequency'],ascending=False)

In [ ]:
attribs = [list(dfs_rndf[i]['Feature']) for i in range(0,9)]
attribs = [attrib for listAttrib in attribs for attrib in listAttrib]
pd.DataFrame(Counter(attribs),index=['Frequency']).transpose().sort_values(by=['Frequency'],ascending=False)

In [ ]:
attribs = [list(dfs_logistic[i]['Feature']) for i in range(0,9)]
attribs += [list(dfs_tree[i]['Feature']) for i in range(0,9)]
attribs += [list(dfs_rndf[i]['Feature']) for i in range(0,9)]
attribs = [attrib for listAttrib in attribs for attrib in listAttrib]
pd.DataFrame(Counter(attribs),index=['Frequency']).transpose().sort_values(by=['Frequency'],ascending=False)

In [ ]:
logisticDf.sort_values(by='10%',inplace=True,ascending=False)
fig = {
    'data' : [
        {'x' : logisticDf.Feature.head(15),'y' : logisticDf['10%'].head(15), 'mode' : 'markers', 'name' : '10%'}
    ]
}
iplot(fig)

In [ ]:
obj1.precision

In [ ]:
classifiers[0].preds

In [5]:
data= CH.getMasterData("../FinalResults/ImgShrRnkListWithTags.csv")  
methods = ['dummy','bayesian','logistic','svm','dtree','random_forests','ada_boost']
kwargsDict = {'dummy' : {'strategy' : 'most_frequent'},
            'bayesian' : {'fit_prior' : True},
            'logistic' : {'penalty' : 'l2'},
            'svm' : {'kernel' : 'rbf','probability' : True},
            'dtree' : {'criterion' : 'entropy'},
            'random_forests' : {'n_estimators' : 10 },
            'ada_boost' : {'n_estimators' : 50 }}

In [7]:
allAttribs = CH.genAllAttribs("../FinalResults/ImgShrRnkListWithTags.csv",'non_sparse',"../data/infoGainsExpt2.csv")
clfObj = CH.buildBinClassifier(data,allAttribs,1-0.5,80,'dtree',kwargsDict['dtree'])
clfObj.runClf()

0

In [10]:
clfObj.precision,clfObj.recall,clfObj.methodName

(0.66666666666666663, 0.7068965517241379, 'dtree')

In [9]:
fpr,tpr,_ = clfObj.roccurve
rocCurve = {}
for i in range(len(fpr)):
    rocCurve[fpr[i]] = tpr[i]
    
pd.DataFrame(rocCurve,index=['tpr']).transpose().iplot()

In [256]:
train_data_fl = "../FinalResults/ImgShrRnkListWithTags.csv"
attribType = 'non_sparse'
infoGainFl = "../data/infoGainsExpt2.csv"
allAttribs = CH.genAllAttribs(train_data_fl,attribType,infoGainFl)
methodName='linear'
train_data= CH.getMasterData(train_data_fl)

In [257]:
import BaseCapsuleClass 
importlib.reload(BaseCapsuleClass)
rgrObj = CH.buildRegrMod(train_data_fl,allAttribs,0.6,methodName,kwargs={'fit_intercept':True})



In [258]:
rgrObj.test_x #691
rgrObj.test_y #691
rgrObj.train_x.to_csv("/tmp/train_x.dump.csv")
rgrObj.train_y #460

rgrObj.runClf()

In [260]:
rgrObj.preds

[48.454463928472251,
 78.711454160744324,
 86.953641660744324,
 38.091182678472251,
 83.927120178472251,
 112.07082916074432,
 44.821361935930327,
 34.609672261518426,
 58.637235410744324,
 60.449942181468941,
 60.738797910744324,
 62.872432678472251,
 77.176404516096227,
 53.687797261518426,
 41.251018337556161,
 54.488797910744324,
 100.50774476211518,
 55.473172910744324,
 41.941998417722061,
 34.250516660744324,
 78.653067181468941,
 50.867704160744324,
 50.464851518976502,
 70.996817181468941,
 65.829143337556161,
 63.523954160744324,
 43.433601518976502,
 40.714851518976502,
 -618253024227314.12,
 34.184110410744324,
 26.934110410744324,
 36.540111935930327,
 33.438016660744324,
 35.551373417722061,
 51.626018337556161,
 88.847878004657105,
 58.739330685930327,
 29.965065504657105,
 85.41366875753738,
 15.465360410744324,
 50.266141660744324,
 101.23933068593033,
 36.057887857081369,
 28.036205685930327,
 53.969266660744324,
 52.196361935930327,
 55.274486935930327,
 24.219266660

In [115]:
errors = [testy[i]-preds[i] for i in range(len(preds)) if abs(testy[i]-preds[i]) <= 100]

In [118]:
sum(errors)/len(errors)

1.2549046579889751

In [119]:
outliers = [testy[i]-preds[i] for i in range(len(preds)) if abs(testy[i]-preds[i]) > 100]

In [120]:
outliers

[-144222989624324.97,
 -1214150026129.988,
 4091148871.587543,
 253665574945685.31,
 880660374527255.38,
 -501903394479.68677,
 6400542402051.9121,
 -103563922197765.53]

In [127]:
readyDict = CH.createDataFlDict(train_data,allAttribs,80,dataMode='train')

In [167]:
CH.trainTestSplitter(readyDict,allAttribs,0.6)

KeyError: "['TARGET'] not in index"

In [132]:
import PopulationEstimatorFromClf as PE

In [136]:
clfObj,_ = PE.trainTestClf("../FinalResults/ImgShrRnkListWithTags.csv",
    "../data/full_gid_aid_ftr_agg.csv",
    'logistic',
    'non_sparse',
    infoGainFl="../data/infoGainsExpt2.csv",clfArgs = {'logistic' : {'penalty' : 'l2'}})

In [138]:
clfObj.train_x.columns

Index(['ground', 'grassy', 'UNKNOWN SEX', 'antelope', 'excellent', 'front',
       'water', 'group', 'walking', 'giraffe_masai', 'big cat', 'lone',
       'hyena', 'way', 'Male', 'path', 'green', 'right', 'brown', 'bushes',
       'tall', 'mother', 'wild', 'herd', 'hay', 'dry', 'drinking', 'frontleft',
       'animal', 'infant', 'poor', 'left', 'brush', 'open', 'road', 'tree',
       'plant', 'giraffe', 'laying', 'good', 'adult', 'adult', 'ok', 'zebra',
       'plain', 'elephant', 'juveniles - one year old', 'pond', 'back', 'dirt',
       'forest', 'junk', 'field', 'backleft', 'looking', 'baby', 'grass',
       'sheep', 'unknown', 'cactus', 'arthropod', 'mountain', 'sky', 'llama',
       'backright', 'mammal', 'outdoor', 'lake', 'hill', 'crossing',
       'UNKNOWN NAME', 'leaf', 'grazing', 'river', 'lush', 'standing',
       'frontright', 'juveniles- two year old', 'deer', 'eating', 'drink',
       'conifer', 'desert', 'Female', 'running', 'zebra_plains'],
      dtype='object')

In [139]:
allAttribs = CH.genAllAttribs(train_data_fl,'non_sparse',None)
train_data= CH.getMasterData(train_data_fl)

# # Create testing data
# test_data= CH.getMasterData(test_data_fl)
# testObj = CH.createDataFlDict(test_data,allAttribs,0.8,'Test')

# testDf =  pd.DataFrame(testObj).transpose()
# attributes = testDf.columns[:len(allAttribs)]
# testDataFeatures = testDf[allAttribs]

In [141]:
readyDict = CH.createDataFlDict(train_data,allAttribs,80,dataMode='train')

In [143]:
pd.DataFrame(readyDict).transpose().head()

,Female,IBEIS_PZ_0001,IBEIS_PZ_0003,IBEIS_PZ_0005,IBEIS_PZ_0007,IBEIS_PZ_0010,IBEIS_PZ_0011,IBEIS_PZ_0012,IBEIS_PZ_0013,IBEIS_PZ_0014,...,standing,tall,tree,unknown,walking,water,way,wild,zebra,zebra_plains
10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1005,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1012,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1016,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1024,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [152]:
from collections import OrderedDict
def createDataFlDict(data,allAttribs,threshold,dataMode ='Train',writeTempFiles=False,ftrs = ['SPECIES','SEX','AGE','QUALITY','VIEW_POINT','INDIVIDUAL_NAME']):
    gidAttribDict = {}

    if dataMode == 'Train':
        fDataKeys= list(filter(lambda x : float(data[x]['Proportion']) >= 80.0 or float(data[x]['Proportion']) <= 20.0,data.keys()))
        fData = {gid: data[gid] for gid in fDataKeys}
    else: # there will be no proportion, unlablelled data
        fData = data

    for gid in fData.keys():
        ftrDict = fData[gid]
        attribDict = OrderedDict.fromkeys(allAttribs,0)

        for ftr in ftrs:
            for itm in set(ftrDict[ftr].split(',')):
                if itm in allAttribs:
                    attribDict[itm] = 1

        # logic for tgs
        for tag in set(literal_eval(ftrDict['tags'])):
            attribDict[tag] = 1

        if dataMode == 'Train':
            attribDict['TARGET'] = 1 if float(ftrDict['Proportion'] ) >= threshold else 0 # Thresholding for the share proportion

        gidAttribDict[gid] = attribDict
    
    if writeTempFiles:
        json.dump(gidAttribDict,open("/tmp/gidAttribDict.json","w"),indent=4)
        pd.DataFrame(gidAttribDict).transpose().to_csv("/tmp/gidAttribDict.csv")
        
    return gidAttribDict

In [153]:
createDataFlDict(train_data,allAttribs,0.8)

{'10': OrderedDict([('giraffe_masai', 1),
              ('zebra_plains', 0),
              ('Male', 0),
              ('Female', 1),
              ('UNKNOWN SEX', 0),
              ('UNKNOWN NAME', 0),
              ('juveniles- two year old', 0),
              ('juveniles - one year old', 0),
              ('unknown', 0),
              ('adult', 1),
              ('infant', 0),
              ('poor', 0),
              ('junk', 0),
              ('good', 0),
              ('excellent', 0),
              ('ok', 1),
              ('right', 0),
              ('backright', 0),
              ('back', 0),
              ('front', 0),
              ('frontleft', 0),
              ('frontright', 0),
              ('backleft', 0),
              ('left', 1),
              ('ground', 0),
              ('grassy', 1),
              ('antelope', 0),
              ('water', 0),
              ('group', 1),
              ('big cat', 0),
              ('lone', 0),
              ('hyena', 0),
            

In [151]:
allAttribs

['giraffe_masai',
 'zebra_plains',
 'Male',
 'Female',
 'UNKNOWN SEX',
 'UNKNOWN NAME',
 'juveniles- two year old',
 'juveniles - one year old',
 'unknown',
 'adult',
 'infant',
 'poor',
 'junk',
 'good',
 'excellent',
 'ok',
 'right',
 'backright',
 'back',
 'front',
 'frontleft',
 'frontright',
 'backleft',
 'left',
 'ground',
 'grassy',
 'antelope',
 'water',
 'group',
 'big cat',
 'lone',
 'hyena',
 'way',
 'path',
 'green',
 'brown',
 'bushes',
 'tall',
 'mother',
 'wild',
 'herd',
 'drinking',
 'dry',
 'desert',
 'animal',
 'hay',
 'brush',
 'open',
 'road',
 'tree',
 'plant',
 'giraffe',
 'laying',
 'zebra',
 'adult',
 'plain',
 'elephant',
 'pond',
 'dirt',
 'forest',
 'field',
 'looking',
 'baby',
 'grass',
 'sheep',
 'cactus',
 'arthropod',
 'mountain',
 'sky',
 'llama',
 'mammal',
 'outdoor',
 'lake',
 'hill',
 'crossing',
 'leaf',
 'grazing',
 'river',
 'lush',
 'standing',
 'deer',
 'eating',
 'drink',
 'conifer',
 'walking',
 'running']

In [261]:
minSplit = 0.2
maxSplit = 0.6

data= CH.getMasterData("../FinalResults/ImgShrRnkListWithTags.csv")  
methods = ['dummy','bayesian','logistic','svm','dtree','random_forests','ada_boost']

kwargsDict = {'dummy' : {'strategy' : 'most_frequent'},
            'bayesian' : {'fit_prior' : True},
            'logistic' : {'penalty' : 'l2'},
            'svm' : {'kernel' : 'rbf','probability' : True},
            'dtree' : {'criterion' : 'entropy'},
            'random_forests' : {'n_estimators' : 10 },
            'ada_boost' : {'n_estimators' : 50 }}

for attribsType in ['sparse','non_sparse','non_zero','abv_mean']:
    print("Classifier training started for %s" %attribsType)  

    allAttribs = CH.genAllAttribs("../FinalResults/ImgShrRnkListWithTags.csv",attribsType,"../data/infoGainsExpt2.csv")
    codes = []
    classifiers = []
    for i in np.arange(minSplit,maxSplit,0.1): # i is the test percent
        clfForRoc = []
        for method in methods:
            clfObj = CH.buildBinClassifier(data,allAttribs,1-i,80,method,kwargsDict[method])
            clfObj.runClf()
            classifiers.append(clfObj)
            clfForRoc.append(clfObj)

Classifier training started for sparse


TypeError: runClf() missing 1 required positional argument: 'self'

In [188]:
for clf in classifiers:
    print(clf)

{'sqerr': 0.4526627218934911, 'methodName': 'dummy', 'abserr': 0.4526627218934911, 'auc': 0.5, 'precision': 0.5473372781065089, 'f1Score': 0.70745697896749526, 'splitPercent': 0.80000000000000004, 'zerooneloss': 0.4526627218934911, 'accScore': 0.5473372781065089, 'recall': 1.0}
{'sqerr': 0.32544378698224852, 'methodName': 'bayesian', 'abserr': 0.32544378698224852, 'auc': 0.70183713124889602, 'precision': 0.67942583732057416, 'f1Score': 0.7208121827411168, 'splitPercent': 0.80000000000000004, 'zerooneloss': 0.32544378698224852, 'accScore': 0.67455621301775148, 'recall': 0.76756756756756761}
{'sqerr': 0.31065088757396447, 'methodName': 'logistic', 'abserr': 0.31065088757396447, 'auc': 0.71711711711711723, 'precision': 0.69047619047619047, 'f1Score': 0.73417721518987344, 'splitPercent': 0.80000000000000004, 'zerooneloss': 0.31065088757396453, 'accScore': 0.68934911242603547, 'recall': 0.78378378378378377}
{'sqerr': 0.4526627218934911, 'methodName': 'svm', 'abserr': 0.4526627218934911, 'au

In [185]:
import BaseCapsuleClass as Base
importlib.reload(Base)

<module 'BaseCapsuleClass' from '/Users/sreejithmenon/Google Drive/Project/AnimalPhotoBias/script/BaseCapsuleClass.py'>